In [1]:
import numpy as np
import pandas as pd
import torch
import re
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/WELFake_Dataset.csv")
print(df.columns)
print(df.shape)
print(type(df.loc[7,"text"]))

text_iterator = df['text']
title_iterator = df['title']

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')
(72134, 4)
<class 'str'>


In [ ]:
# with stemming and removing stop words with nltk
# not in use
ps = PorterStemmer()

def process_text(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            # stem the words
            new_t = []
            for w in t:
              new_word = ps.stem(w)
              if new_word not in (stop_words):
                new_t.append(new_word)
            # rejoin tokenized string
            t = " ".join(new_t)
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text(text_iterator)
title_processed = process_text(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

In [ ]:
# just stemming of words
# not in use
def process_text2(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9_.,!\"\'/$]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            # stem the words
            new_t = []
            for w in t:
              new_word = ps.stem(w)
              new_t.append(new_word)
            # rejoin tokenized string
            t = " ".join(new_t)
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text2(text_iterator)
title_processed = process_text2(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

In [4]:
# no stemming and removing stop words
def process_text3(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9_.,!\"\'/$]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            # join back the string
            new_t = []
            for w in t:
              new_t.append(w)
            # rejoin tokenized string
            t = " ".join(new_t)
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text3(text_iterator)
title_processed = process_text3(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

72134
72134
Now, most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change. Loretta Lynch aka Eric Holder in a skirt


In [15]:
merged_title_text = []
for i,v in enumerate(title_processed):
    if (v == None):
        merge = text_processed[i]
    elif (text_processed[i]==None):
        merge = v
    else:
        merge = v + " " + text_processed[i]
    merged_title_text.append(merge)

print(merged_title_text[2])

UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE PEACEFUL PROTESTERS In Her Home State Of North Carolina VIDEO Now, most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change. Loretta Lynch aka Eric Holder in a skirt


In [16]:
df_output = pd.DataFrame(columns=["title", "text", "merged", "class"])
df_output["title"] = title_processed
df_output["text"] = text_processed
df_output["merged"] = merged_title_text
df_output["class"] = df["label"]

In [21]:
# split test and training dataset
x_train, x_test, y_train, y_test = train_test_split(df_output["merged"], df_output["class"], test_size=0.2, shuffle=True, random_state=23)

print(x_train[0:3])
print(y_train[0:3])
print(len(x_train))
print(x_train[57705])


27099    Fired Racist Breitbart Writer Says She Needs Y...
30389    Here Are Six Miracle Drugs Big Pharma Now Regr...
12462    WHOA! Republican Governor Just Compared Donald...
Name: merged, dtype: object
27099    1
30389    1
12462    1
Name: class, dtype: int64
57707
On Trump s victory Gilad Atzmon It occurred to me in recent years that the act of being progressive is not a political position but rather a mental state. The incapacity of the entire American progressive and left establishment to foresee Trump s landslide victory suggests that we are dealing with people who are institutionally detached Just three days ahead of the presidential elections, the Huffington Post pathetically criticised star pollster Nate Silver of Unskewing Polls in Trump s direction, for suggesting that a Trump victory was realistic. Ryan Grim wrote HuffPost Pollster is giving Clinton a 98 percent chance of winning, and The New York Times model at The Upshot puts her chances at 85 percent. There is one outl

In [8]:
# get text length
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

# print((get_text_length(x_test[5:15])))


In [9]:
# get pos of words
stopWords = stopwords.words('english')
def get_pos_words(x):
  n_ctr, a_ctr, v_ctr, r_ctr, others, stopwords = 0, 0, 0, 0, 0, 0
  classified = ""
  counters = []
  output = []
  for t in x:
    counters = []
    n_ctr, a_ctr, v_ctr, r_ctr, others, stopwords = 0, 0, 0, 0, 0, 0
    for w in t.split(" "):
      if w not in stopWords:
        if wn.synsets(w):
          classified = wn.synsets(w)[0].pos()
          if classified == "n": n_ctr += 1
          elif classified == "a": a_ctr += 1
          elif classified == "v": v_ctr += 1
          elif classified == "r": r_ctr += 1
        else: others += 1
      else: stopwords += 1
    counters.append(n_ctr/len(t.split(" ")))
    counters.append(a_ctr/len(t.split(" ")))
    counters.append(v_ctr/len(t.split(" ")))
    counters.append(r_ctr/len(t.split(" ")))
    counters.append(others/len(t.split(" ")))
    counters.append(stopwords/len(t.split(" ")))
    output.append(counters)
  return np.array(output)

# print(get_pos_words(x_test[5:15]))

In [10]:
# get no. of capital letters
def get_capital_letters(x):
  counter = 0
  total = []
  for t in x:
    number = []
    counter = 0
    for l in t:
      if l.isupper(): counter+=1
    number.append(counter/len(t))
    total.append(number)
  return np.array(total)

# print(get_capital_letters(x_test[5:10]))

In [30]:
# get no. of short/long sentences
# the number of words that determine whether a sentence is long (30), gotten from
# https://languagetool.org/insights/post/sentence-length/#:~:text=Sentences%20are%20usually%20between%2015,should%20be%20considered%20the%20maximum.
def get_long_short_sentences(x):
  longg, short = 0, 0
  total = []
  for t in x:
    longg, short = 0, 0
    count = []
    sentence_list = nltk.sent_tokenize(t)
    for s in sentence_list:
      no_of_words = s.split(" ")
      if len(no_of_words) > 30:
        longg += 1
      else:
        short += 1
    if len(sentence_list) != 0:
      count.append(longg/len(sentence_list))
      count.append(short/len(sentence_list))
    else:
      count.append(0)
      count.append(0)
    total.append(count)
  return np.array(total)


# print(get_long_short_sentences(x_train).shape)


In [12]:
# get no. of special characters
# what is considered a special character taken from
# https://www.webopedia.com/definitions/special-character/#:~:text=A%20special%20character%20is%20one,marks%20are%20also%20special%20characters.
def get_special_characters(x):
  total = []
  for t in x:
    characters = re.sub('[\w]+','',t)
    characters = re.sub(r" +", "", characters)
    if len(t) != 0:
      total.append([len(characters)/len(t)])
    else:
      total.append([0])
  return np.array(total)

# print(get_special_characters(x_test[5:10]))


In [13]:
def get_accuracy(predicted, test):
  num_correct = 0
  for i in range(len(test)):
    if predicted[i] == test[i]:
      num_correct +=  1
  return num_correct/len(test)

In [ ]:
# normal bag of words model
classifier = Pipeline([
    ('vectorizer', CountVectorizer(min_df=0, max_df=1.0, strip_accents='ascii', stop_words=stopWords, ngram_range=(1,2), max_features=500)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())])
classifier.fit(x_train, y_train.ravel())

predicted = classifier.predict(x_test)

print(get_accuracy(predicted, np.array(y_test)))

0.8543009634712692


In [ ]:
# bag of words model with text length
classifier2 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier2.fit(x_train, y_train.ravel())
predicted2 = classifier2.predict(x_test)

print(get_accuracy(predicted2, np.array(y_test)))

12362
0.8568655992236778


In [ ]:
# normal bag of words model with length and pos
classifier3 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier3.fit(x_train, y_train.ravel())
predicted3 = classifier3.predict(x_test)

print(get_accuracy(predicted3, np.array(y_test)))

12333
0.8548554793096278


In [ ]:
# normal bag of words model with length and pos and proportion of capital letters
classifier4 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier4.fit(x_train, y_train.ravel())
predicted4 = classifier4.predict(x_test)

print(get_accuracy(predicted4, np.array(y_test)))

12333
0.8548554793096278


In [ ]:
# normal bag of words model and proportion of capital letters
classifier5 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier5.fit(x_train, y_train.ravel())
predicted5 = classifier5.predict(x_test)

print(get_accuracy(predicted5, np.array(y_test)))

12511
0.8671934567131073


In [ ]:
# normal bag of words model and proportion of capital letters and pos
classifier6 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.85)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier6.fit(x_train, y_train.ravel())
predicted6 = classifier6.predict(x_test)

print(get_accuracy(predicted6, np.array(y_test)))

12515
0.8674707146322866


In [ ]:
# normal bag of words model and proportion of capital letters and pos
classifier7 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.75)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier7.fit(x_train, y_train.ravel())
predicted7 = classifier6.predict(x_test)

print(get_accuracy(predicted7, np.array(y_test)))

12525
0.868163859430235


In [33]:
# normal bag of words model and proportion of capital letters and pos
classifier8 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ])),
        ('sls', Pipeline([
            ('count', FunctionTransformer(get_long_short_sentences, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier8.fit(x_train, y_train.ravel())

predicted8 = classifier8.predict(x_test)

print(get_accuracy(predicted8, np.array(y_test)))

0.8671241422333126


In [34]:
# normal bag of words model and proportion of capital letters and pos
classifier9 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ])),
        ('sls', Pipeline([
            ('count', FunctionTransformer(get_long_short_sentences, validate=False)),
        ])),
        ('sc', Pipeline([
            ('count', FunctionTransformer(get_special_characters, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier9.fit(x_train, y_train.ravel())

predicted9 = classifier9.predict(x_test)

print(get_accuracy(predicted9, np.array(y_test)))

0.8671241422333126


In [38]:
# normal bag of words model and proportion of capital letters and pos
classifier10 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.75)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ])),
        ('sls', Pipeline([
            ('count', FunctionTransformer(get_long_short_sentences, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier10.fit(x_train, y_train.ravel())

predicted10 = classifier10.predict(x_test)

print(get_accuracy(predicted10, np.array(y_test)))

0.8678866015110557
